# Malaria Detection

Malaria is a life-threatening disease caused by parasites that are transmitted to people through the bites of infected female Anopheles mosquitoes. It is preventable and curable.
* In 2017, there were an estimated 219 million cases of malaria in 90 countries.
* Malaria deaths reached 435 000 in 2017.
* The WHO African Region carries a disproportionately high share of the global malaria burden. In 2017, the region was home to 92% of malaria cases and 93% of malaria deaths.

Malaria is caused by Plasmodium parasites. The parasites are spread to people through the bites of infected female Anopheles mosquitoes, called "malaria vectors." There are 5 parasite species that cause malaria in humans, and 2 of these species – P. falciparum and P. vivax – pose the greatest threat.


***Diagnosis of malaria can be difficult***:

* Where malaria is not endemic any more (such as in the United States), health-care providers may not be familiar with the disease. Clinicians seeing a malaria patient may forget to consider malaria among the potential diagnoses and not order the needed diagnostic tests. Laboratorians may lack experience with malaria and fail to detect parasites when examining blood smears under the microscope.
* Malaria is an acute febrile illness. In a non-immune individual, symptoms usually appear 10–15 days after the infective mosquito bite. The first symptoms – fever, headache, and chills – may be mild and difficult to recognize as malaria. If not treated within 24 hours, P. falciparum malaria can progress to severe illness, often leading to death.

***Microscopic Diagnosis***

Malaria parasites can be identified by examining under the microscope a drop of the patient’s blood, spread out as a “blood smear” on a microscope slide. Prior to examination, the specimen is stained  to give the parasites a distinctive appearance. This technique remains the gold standard for laboratory confirmation of malaria. However, it depends on the quality of the reagents, of the microscope, and on the experience of the laboratorian.

![malaria](https://cdn1.sph.harvard.edu/wp-content/uploads/2015/03/Malaria-cells_CDC.jpg)

references :
* [1] https://www.who.int/news-room/fact-sheets/detail/malaria 
* [2] https://www.cdc.gov/malaria/diagnosis_treatment/diagnosis.html 


### Install dependencies

In [ ]:
%%bash
pip install git+https://github.com/fastai/fastai2
pip install torch torchvision feather-format pillow=='6.2.0' kornia pyarrow --upgrade   > /dev/null

## EDA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fastai2.data.all import *
from fastai2.vision.data import *
from fastai2.vision.all import *

from sklearn.metrics import auc, roc_curve, precision_recall_curve, classification_report

import os
print(os.listdir("../input/cell_images/cell_images/"))

## Dataset

In [ ]:
path=Path('../input/cell_images/cell_images/')
path.ls()

In [ ]:
splitter = RandomSplitter()
item_tfms = [Resize(224)]
batch_tfms=[*aug_transforms(flip_vert=True,max_zoom=1.2, max_warp=0), Normalize.from_stats(*imagenet_stats)]

In [ ]:
data_block = DataBlock(blocks=[ImageBlock, CategoryBlock],
                  get_items=get_image_files,
                  get_y=parent_label,
                  splitter=splitter,
                  item_tfms=item_tfms,
                  batch_tfms=batch_tfms)

In [ ]:
data = data_block.dataloaders(path, bs=64)
data.show_batch()

## Model XResNet34

In [ ]:
learn = cnn_learner(data, xresnet34, metrics=[accuracy]).to_fp16()


In [ ]:
learn.fine_tune(15)

In [ ]:
learn.save('stage-1')

## Confusion Matrix

In [ ]:

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,8))

In [ ]:
preds, y_true=learn.get_preds()
y_true.shape, preds.shape

In [ ]:
y_true=y_true.numpy() 
preds=np.argmax(preds.numpy(), axis=-1)
y_true.shape, preds.shape

In [ ]:
classes = list(data.vocab)
report = classification_report(y_true, preds, target_names=classes)
print(report)

In [ ]:
learn.show_results()

In [ ]:
nan